<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задание-1" data-toc-modified-id="Задание-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задание 1</a></span></li><li><span><a href="#Задание-2" data-toc-modified-id="Задание-2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Задание 2</a></span></li><li><span><a href="#Задание-3" data-toc-modified-id="Задание-3-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Задание 3</a></span></li><li><span><a href="#Задание-4" data-toc-modified-id="Задание-4-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Задание 4</a></span></li><li><span><a href="#Задание-5" data-toc-modified-id="Задание-5-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Задание 5</a></span></li><li><span><a href="#Задание-6" data-toc-modified-id="Задание-6-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Задание 6</a></span></li></ul></div>

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.docstore.wikipedia import Wikipedia
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.agents.react.base import DocstoreExplorer
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

In [3]:
from langchain.utilities import OpenWeatherMapAPIWrapper

In [5]:
chat = ChatOpenAI(temperature=0.1)

## Задание 1
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. Стандартная роза высотой 25—35 см, красная и на ней не менее 6 бутонов. Роза Фёдора красного цвета,  25 см высотой и на ней 5 бутонов. Роза Василия розовая, 24 см высотой и на ней 7 бутонов. Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. С помощью ChatGPT рассуди конкурс. 

In [6]:
prompt_task1_v1 = """
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. 
На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. 
Стандартная роза высотой 25—35 см, красная и на ней не менее 6 бутонов. 
Роза Фёдора красного цвета,  25 см высотой и на ней 5 бутонов. 
Роза Василия розовая, 24 см высотой и на ней 7 бутонов. 
Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. 
Кто должен выиграть?
"""

In [7]:
print(chat([HumanMessage(content=prompt_task1_v1)]).content)

Выиграть должна роза Василия, так как она соответствует всем стандартам: высота 24 см (входит в диапазон 25-35 см), на ней 7 бутонов (больше, чем требуется) и она красиво розового цвета. Роза Фёдора не соответствует высоте, а роза Ивана не подходит из-за наличия помидоров.


Видим, что ChatGPT пропускает, что роза должна быть красная, и считает, что 24 см лежит в интервале 25-35см (а 25 см не лежит!). Попробуем переформулировать промпт.

In [8]:
prompt_task1_v2 = """
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. 
На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. 
Стандартная роза должна быть высотой 25—35 см, красного цвета и иметь не менее 6 бутонов. 
Роза Фёдора красного цвета, 25 см высотой и на ней 5 бутонов. 
Роза Василия розовая, 24 см высотой и на ней 7 бутонов. 
Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. 
Кто должен выиграть?
"""
print(chat([HumanMessage(content=prompt_task1_v2)]).content)

Ни одна из роз не соответствует стандартам, так как у каждой из них есть недостаток: у розы Фёдора не хватает бутона, у розы Василия недостаточная высота, а у розы Ивана не соответствует цвет. Следовательно, никто не должен выиграть.


Немного переформулировала, какой должна быть стандартная роза, и ChatGPT увидел, что цвет тоже важен, но теперь считает, что роза Ивана не соответствует по цвету. Нужный ответ получен, но рассуждения модели не совсем верные. Хочу отметить, что когда я давала тот же промпт в другой день, то модель давала верный ответ и правильно объясняла свое решение.

Можно предложить few-shot промпт с примером chain_of_thought reasoning.

In [9]:
template = """
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. 
Стандартная роза высотой 25—35 см, красного цвета и на ней не менее 6 бутонов. 
Определи, кто должен выиграть.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

example_prompt = """
На конкурсе заявлены роза Виталия, роза Анны и роза Михаила.
Роза Виталия желтая, высотой 28 см, имеет 7 бутонов.
Роза Анны красного цвета, высотой 26 см, имеет 8 бутонов.
Роза Михаила красного цвета, имеет 9 бутонов, высотой 30 см, и на ней растет редиска.
Кто должен выиграть?
"""

example_ai_response = """
Роза Виталия подходит по высоте и по количеству бутонов, но не подходит по цвету. 
Роза Анны подходит по цвету, высоте и колчеству бутонов. 
Роза Михаила подходит по цвету, количеству бутонов, высоте, но не подходит, потому что на ней растет редиска.
Роза Анны соответствует всем стандартам, поэтому она должна выиграть.
"""

example_human = HumanMessagePromptTemplate.from_template(example_prompt)
example_ai = AIMessagePromptTemplate.from_template(example_ai_response)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human, example_ai, human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)
prompt_task1_v3 = """
На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. 
Роза Фёдора красного цвета,  25 см высотой и на ней 5 бутонов. 
Роза Василия розовая, 24 см высотой и на ней 7 бутонов. 
Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. 
Кто должен выиграть?
"""
print(chain.run(prompt_task1_v3))

Роза Фёдора не подходит по высоте и количеству бутонов.
Роза Василия не подходит по высоте, хотя подходит по количеству бутонов и цвету.
Роза Ивана подходит по высоте, количеству бутонов и цвету, но не подходит, потому что на ней растут помидоры.
Ни одна из роз не соответствует всем стандартам, поэтому никто не должен выиграть.


Ответ получен правильный, но рассуждения неверные: модель делает неправильный вывод о цвете для розы Василия и неправильный вывод о высоте для розы Федора. Переформулируем промпт.

In [10]:
prompt_task1_v4 = """
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. 
Стандартная роза не ниже 25 см, но не выше 35 см, красного цвета и на ней не менее 6 бутонов. 

Q: На конкурсе заявлены роза Виталия, роза Анны и роза Михаила.
Роза Виталия желтая, высотой 29 см, имеет 7 бутонов.
Роза Анны красного цвета, высотой 26 см, имеет 8 бутонов.
Роза Михаила красного цвета, имеет 9 бутонов, высотой 30 см, и на ней растет редиска.
Кто должен выиграть?
A: Роза Виталия подходит по высоте (29 см не ниже 25 см, но не выше 35 см) и количеству бутонов (7 бутонов), но не подходит по цвету (желтый вместо красного), поэтому не может быть победителем. 
Роза Анны подходит по цвету (красный), высоте (26 см не ниже 25 см и не выше 35 см) и количеству бутонов (8 бутонов). 
Роза Михаила подходит по цвету (красный), высоте (30 см лежит в интервале 25-35 см) и количеству бутонов (9 бутонов), но не подходит, потому что на ней растет редиска, поэтому она не может быть победителем.
Роза Анны соответствует всем стандартам, поэтому она должна выиграть.

Q: На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. 
Роза Фёдора красного цвета, 25 см высотой и на ней 5 бутонов. 
Роза Василия розовая, 24 см высотой и на ней 7 бутонов. 
Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. 
Кто должен выиграть?
"""
print(chat([HumanMessage(content=prompt_task1_v4)]).content)

A: Ни одна из роз не соответствует всем стандартам. Роза Фёдора не подходит по высоте (25 см ниже минимальной высоты 25 см), количеству бутонов (5 бутонов меньше минимального количества 6 бутонов) и розового цвета вместо красного. Роза Василия не подходит по высоте (24 см ниже минимальной высоты 25 см) и цвету (розовый вместо красного). Роза Ивана не подходит из-за наличия помидоров на растении. Победитель не определен, так как нет роз, которые соответствуют всем стандартам.


Удалось заставить chatGPT сделать правильный вывод о цвете розы Василия, но снова пострадал цвет розы Федора, и модель упорно считает, что роза Федора не подходит по высоте. Немного изменим наш пример.

In [11]:
prompt_task1_v5 = """
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. 
Стандартная роза не ниже 25 см, но не выше 35 см, красного цвета и на ней не менее 6 бутонов. 

Q: На конкурсе заявлены роза Виталия, роза Анны и роза Михаила.
Роза Виталия желтая, высотой 29 см, имеет 7 бутонов.
Роза Анны красного цвета, высотой 25 см, имеет 8 бутонов.
Роза Михаила красного цвета, имеет 9 бутонов, высотой 30 см, и на ней растет редиска.
Кто должен выиграть?
A: Роза Виталия подходит по высоте (29 см больше 25 см и меньше 35 см) и количеству бутонов (7 бутонов), но не подходит по цвету (желтый вместо красного), поэтому не может быть победителем. 
Роза Анны подходит по цвету (красный), высоте (25 см не меньше 25 см) и количеству бутонов (8 бутонов). 
Роза Михаила подходит по цвету (красный), высоте (30 см больше 25 см и меньше 35 см) и количеству бутонов (9 бутонов), но не подходит, потому что на ней растет редиска, поэтому она не может быть победителем.
Роза Анны соответствует всем стандартам, поэтому она должна выиграть.

Q: На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. 
Роза Фёдора красного цвета, 25 см высотой и на ней 5 бутонов. 
Роза Василия розовая, 24 см высотой и на ней 7 бутонов. 
Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. 
Кто должен выиграть?
"""
print(chat([HumanMessage(content=prompt_task1_v5)]).content)

A: Ни одна из роз не соответствует всем стандартам. Роза Фёдора не подходит по высоте (меньше 25 см), роза Василия не подходит по высоте (меньше 25 см) и цвету (розовый вместо красного), а роза Ивана не подходит из-за наличия помидоров на растении. Победитель не определен.


Снова провал с высотой розы Фёдора.<br>
Уберем пример, но опишем последовательность действий, которую должен выполнить chatGPT, чтобы дать верный результат.

In [12]:
prompt_task1_v6 = """
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. 
Стандартная роза не ниже 25 см, но не выше 35 см, красного цвета и на ней не менее 6 бутонов. 
На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. 
Роза Фёдора красного цвета, 25 см высотой и на ней 5 бутонов. 
Роза Василия розовая, 24 см высотой и на ней 7 бутонов. 
Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. 
Определи стандартные параметры роз. 
Оцени соответствие роз, заявленных в конкурсе, стандартным параметрам.
Какая роза должна выиграть?
"""
print(chat([HumanMessage(content=prompt_task1_v6)]).content)

Стандартные параметры роз: высота от 25 до 35 см, красного цвета и на ней не менее 6 бутонов.

Роза Фёдора не соответствует стандартам, так как ее высота 25 см, а минимальная высота для стандартной розы - 25 см, и на ней 5 бутонов, а минимальное количество бутонов для стандартной розы - 6.

Роза Василия не соответствует стандартам, так как ее высота 24 см, а минимальная высота для стандартной розы - 25 см.

Роза Ивана не соответствует стандартам, так как на ней растут помидоры.

Ни одна из заявленных роз не соответствует стандартам, поэтому никакая роза не должна выиграть.


Снова получаем правильный ответ, но модель  сравнивает 25 с 25 и получает, что стандартная роза не может иметь высоту 25 см :( И опять забывает про цвет!

In [13]:
prompt_task1_height = """
Если заявлен интервал не менее 25 см и не более 35 см, то входит ли 25 см в этот интервал?
"""
print(chat([HumanMessage(content=prompt_task1_height)]).content)

Да, 25 см входит в интервал не менее 25 см и не более 35 см.


In [14]:
prompt_task1_v7 = """
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. 
Стандартная роза не ниже 25 см, но не выше 35 см, красного цвета и на ней не менее 6 бутонов. 
На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. 
Роза Фёдора красного цвета, 25 см высотой и на ней 5 бутонов. 
Роза Василия розовая, 24 см высотой и на ней 7 бутонов. 
Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. 

Сначала выведи параметры стандартной розы.
Затем сравни параметры роз, заявленных на конкурс, с стандартными параметрами. \
Опиши, по каким параметрам розы подходят и не подходят.
Могут ли розы с такими параметрами участвовать в конкурсе?
Какая роза должна выиграть?
"""
print(chat([HumanMessage(content=prompt_task1_v7)]).content)

Параметры стандартной розы: высота не ниже 25 см, но не выше 35 см, красного цвета и на ней не менее 6 бутонов.

Роза Фёдора подходит по цвету, но не подходит по высоте (ниже минимальной) и по количеству бутонов (меньше минимального). Не может участвовать в конкурсе.

Роза Василия не подходит по цвету, но подходит по высоте и количеству бутонов. Может участвовать в конкурсе.

Роза Ивана подходит по цвету и количеству бутонов, но не подходит по высоте (выше максимальной) и, к тому же, на ней растут помидоры. Не может участвовать в конкурсе.

Выиграть должна роза Василия, так как она единственная подходит по высоте и количеству бутонов, а цвет не является обязательным параметром.


Рассуждения и ответ неправильные.

In [15]:
prompt_task1_v8 = """
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. 
Стандартная роза не ниже 25 см, но не выше 35 см, красного цвета и на ней не менее 6 бутонов. 
На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. 
Роза Фёдора красного цвета, 25 см высотой и на ней 5 бутонов. 
Роза Василия розовая, 24 см высотой и на ней 7 бутонов. 
Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. 

Сначала выведи параметры стандартной розы:
- минимальная и максимальная высота
- цвет
- количество бутонов
Затем сравни параметры роз, заявленных на конкурс, с каждым из стандартных параметров. \
Опиши, по каким параметрам розы подходят и не подходят.
Могут ли розы с такими параметрами участвовать в конкурсе?
Какая роза должна выиграть?
"""
print(chat([HumanMessage(content=prompt_task1_v8)]).content)

Параметры стандартной розы:
- Минимальная высота: 25 см
- Максимальная высота: 35 см
- Цвет: красный
- Количество бутонов: не менее 6

Роза Фёдора:
- Высота: 25 см (соответствует стандарту)
- Цвет: красный (соответствует стандарту)
- Количество бутонов: 5 (не соответствует стандарту)

Роза Василия:
- Высота: 24 см (не соответствует стандарту)
- Цвет: не красный (не соответствует стандарту)
- Количество бутонов: 7 (соответствует стандарту)

Роза Ивана:
- Высота: 28 см (соответствует стандарту)
- Цвет: красный (соответствует стандарту)
- Количество бутонов: 8 (соответствует стандарту)
- Наличие помидоров: не соответствует стандарту

Роза Фёдора не подходит по количеству бутонов, роза Василия не подходит по высоте и цвету, роза Ивана не подходит по наличию помидоров. 

Участвовать в конкурсе могут только розы, которые соответствуют всем стандартным параметрам. 

Выиграть должна роза Ивана, так как она соответствует большинству стандартных параметров.


Правильно оценили соответствие роз стандарту, но сделали неправильный выбор. Наверно, дело в том, что мы сами зафиксировали высоту, цвет и количество бутонов как стандартные параметры, поэтому помидоры на розе Ивана хотя и определились как аномалия, но не стали решающим фактором.

In [16]:
prompt_task1_v9 = """
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. 
Стандартная роза не ниже 25 см, но не выше 35 см, красного цвета и на ней не менее 6 бутонов. 
На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. 
Роза Фёдора красного цвета, 25 см высотой и на ней 5 бутонов. 
Роза Василия розовая, 24 см высотой и на ней 7 бутонов. 
Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. 

Сначала выведи параметры стандартной розы:
- минимальная и максимальная высота
- цвет
- количество бутонов
Затем сравни параметры роз, заявленных на конкурс, с каждым из стандартных параметров. \
Опиши, по каким параметрам розы подходят и не подходят.
Могут ли розы с такими параметрами участвовать в конкурсе?
Если среди роз есть соответствующие стандартам, то какая роза должна выиграть?
"""
print(chat([HumanMessage(content=prompt_task1_v9)]).content)

Параметры стандартной розы:
- Минимальная высота: 25 см
- Максимальная высота: 35 см
- Цвет: красный
- Количество бутонов: не менее 6

Роза Фёдора:
- Высота: соответствует стандартам
- Цвет: соответствует стандартам
- Количество бутонов: не соответствует стандартам

Роза Василия:
- Высота: не соответствует стандартам
- Цвет: не соответствует стандартам
- Количество бутонов: соответствует стандартам

Роза Ивана:
- Высота: соответствует стандартам
- Цвет: соответствует стандартам
- Количество бутонов: соответствует стандартам, но растут помидоры

Только роза Фёдора не подходит по параметрам и не может участвовать в конкурсе. Роза Василия подходит только по количеству бутонов, а роза Ивана подходит по высоте и цвету, но не подходит по наличию помидоров.

Следовательно, ни одна из роз не соответствует полностью стандартам и не может выиграть конкурс.


Получен **правильный ответ**, правильно определено соответствие параметрам, но следующее предложение - неправда: "Только роза Фёдора не подходит по параметрам и не может участвовать в конкурсе."

In [17]:
prompt_task1_v10 = """
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. 
Стандартная роза имеет высоту не ниже 25 см, но не выше 35 см, красного цвета и на ней не менее 6 бутонов. 
На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. 
Роза Фёдора красного цвета, 25 см высотой и на ней 5 бутонов. 
Роза Василия розовая, 24 см высотой и на ней 7 бутонов. 
Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. 

Сначала выведи параметры стандартной розы.
Затем сравни параметры роз, заявленных на конкурс, с каждым из стандартных параметров. \
Опиши, по каким параметрам розы подходят и не подходят.
Могут ли розы с такими параметрами участвовать в конкурсе?
Если среди роз есть соответствующие стандартам, то какая роза должна выиграть?
"""
print(chat([HumanMessage(content=prompt_task1_v10)]).content)

Параметры стандартной розы: высота 25-35 см, красного цвета, на ней не менее 6 бутонов.

Роза Фёдора подходит по цвету, но не подходит по высоте (меньше минимальной) и по количеству бутонов (меньше минимального). Не может участвовать в конкурсе.

Роза Василия не подходит по цвету, но подходит по высоте и количеству бутонов. Может участвовать в конкурсе.

Роза Ивана подходит по цвету и количеству бутонов, но не подходит по высоте (выше максимальной) и, кроме того, на ней растут помидоры, что является нестандартным. Не может участвовать в конкурсе.

Только роза Василия соответствует всем стандартам и может участвовать в конкурсе. Она должна выиграть.


Опять неправильные рассуждения и неправильный вывод.

In [19]:
prompt_task1_v11 = """
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. 
Стандартная роза имеет высоту не ниже 25 см, но не выше 35 см, красного цвета и на ней не менее 6 бутонов. 
На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. 
Роза Фёдора красного цвета, 25 см высотой и на ней 5 бутонов. 
Роза Василия розовая, 24 см высотой и на ней 7 бутонов. 
Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. 

Сначала выведи параметры стандартной розы.
Оцени соответствие каждой из заявленных роз стандарту.
Могут ли эти розы участвовать в конкурсе?
Если среди роз есть соответствующие стандартам, то какая роза должна выиграть?
"""
print(chat([HumanMessage(content=prompt_task1_v11)]).content)

Параметры стандартной розы: высота 25-35 см, красного цвета, не менее 6 бутонов.

Роза Фёдора не соответствует стандарту, так как у неё только 5 бутонов.
Роза Василия не соответствует стандарту, так как её высота меньше 25 см.
Роза Ивана не соответствует стандарту, так как на ней растут помидоры.

Ни одна из заявленных роз не может участвовать в конкурсе, так как они не соответствуют стандартам.

Никакая из заявленных роз не может выиграть, так как они не соответствуют стандартам.


Вроде бы получили **правильный ответ**, хотя роза Василия не соответствует не только по высоте, но и по цвету. Поймет ли это модель, если несоответствие будет только по цвету?

In [20]:
prompt_task1_v11_2 = """
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. 
Стандартная роза имеет высоту не ниже 25 см, но не выше 35 см, красного цвета и на ней не менее 6 бутонов. 
На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. 
Роза Фёдора красного цвета, 25 см высотой и на ней 5 бутонов. 
Роза Василия розовая, 26 см высотой и на ней 7 бутонов. 
Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. 

Сначала выведи параметры стандартной розы.
Оцени соответствие каждой из заявленных роз стандарту.
Могут ли эти розы участвовать в конкурсе?
Если среди роз есть соответствующие стандартам, то какая роза должна выиграть?
"""
print(chat([HumanMessage(content=prompt_task1_v11_2)]).content)

Параметры стандартной розы: высота 25-35 см, красного цвета, на ней не менее 6 бутонов.

Роза Фёдора не соответствует стандарту, так как у неё всего 5 бутонов.
Роза Василия соответствует стандарту, так как её высота 26 см, на ней 7 бутонов, хоть и не красного цвета.
Роза Ивана не соответствует стандарту, так как на ней растут помидоры.

Только роза Василия может участвовать в конкурсе.

Роза Василия должна выиграть, так как она единственная соответствует стандарту.


Как я и думала, цвет почему-то не воспринимается как обязательный параметр.

In [21]:
prompt_task1_v11_3 = """
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. 
Роза является стандартной, если она красная, высотой 25-35 см и имеет не менее 6 бутонов.

На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. 
Роза Фёдора красного цвета, 25 см высотой и на ней 5 бутонов. 
Роза Василия розовая, 26 см высотой и на ней 7 бутонов. 
Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. 

Сначала выведи параметры стандартной розы.
Выведи параметры роз, заявленных на конкурс. 
Оцени соответствие каждой из них стандарту.
Если среди заявленных роз есть стандартные, то какая роза должна выиграть?
"""
print(chat([HumanMessage(content=prompt_task1_v11_3)]).content)

Параметры стандартной розы: красная, высотой 25-35 см, имеет не менее 6 бутонов.

Параметры заявленных роз:
- Роза Фёдора: красная, 25 см высотой, на ней 5 бутонов. Не соответствует стандарту.
- Роза Василия: розовая, 26 см высотой, на ней 7 бутонов. Не соответствует стандарту.
- Роза Ивана: красная, 28 см высотой, на ней 8 бутонов. Не соответствует стандарту из-за того, что на ней растут помидоры.

Ни одна из заявленных роз не соответствует стандарту, поэтому никакая из них не должна выиграть.


Теперь вернемся к заданным в задании характеристикам роз с новым вариантом промпта.

In [22]:
prompt_task1_v12 = """
В городе проходит конкурс роз, каждое растение оценивают по соответствию стандартам, нестандартные розы не участвуют. 
Роза является стандартной, если она красная, высотой 25-35 см и имеет не менее 6 бутонов.

На конкурсе заявлены роза Фёдора, роза Василия и роза Ивана. 
Роза Фёдора красного цвета, 25 см высотой и на ней 5 бутонов. 
Роза Василия розовая, 24 см высотой и на ней 7 бутонов. 
Роза Ивана красная, 28 см высотой, 8 бутонов и растут помидоры. 

Сначала выведи параметры стандартной розы.
Выведи параметры роз, заявленных на конкурс. 
Оцени соответствие каждой из них стандарту.
Если среди заявленных роз есть стандартные, то какая роза должна выиграть?
"""
print(chat([HumanMessage(content=prompt_task1_v12)]).content)

Параметры стандартной розы: красная, высотой 25-35 см, имеет не менее 6 бутонов.

Параметры заявленных роз:
- Роза Фёдора: красная, 25 см высотой, на ней 5 бутонов. Не соответствует стандарту.
- Роза Василия: розовая, 24 см высотой, на ней 7 бутонов. Не соответствует стандарту.
- Роза Ивана: красная, 28 см высотой, на ней 8 бутонов. Не соответствует стандарту из-за того, что на ней растут помидоры.

Ни одна из заявленных роз не соответствует стандарту, поэтому никакая из них не должна выиграть. Возможно, конкурс будет отменен из-за отсутствия стандартных роз.


**Ответ получен**.

## Задание 2
Составь план действий, чтобы с помощью ChatGPT ответить на следующий вопрос: идёт ли сейчас дождь в городе, где находится самый знаменитый фонтан в стиле барокко? 


Для того чтобы ответить на этот вопрос, нам нужно, чтобы ChatGPT обращался к внешним источникам информации, например к поиску. Сделать это можно по API. Должны быть последовательно получены ответы на следующие вопросы:
1. Какой самый знаменитый фонтан в стиле барокко?
2. Где находится этот фонтан?
3. Какая сейчас погода в этом городе?
4. Идет ли в этом городе сейчас дождь?

При помощи библиотеки `Langchain` можно написать ReAct агента, который сам выстроит последовательность действий и будет обращаться к поиску по API (например, с помощью `SerpAPI`).

У меня нет ключа к `SerpAPI`, но есть ключ к `OpenWeatherAPI` и доступ к википедии, поэтому я попробовала ответить на этот вопрос с помощью доступных средств.

In [35]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [36]:
docstore=DocstoreExplorer(Wikipedia())
weather = OpenWeatherMapAPIWrapper()

tools = [
  Tool(
    name="Search",
    func=docstore.search,
    description="Search for a term in the docstore.",
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup,
    description="Lookup a term in the docstore.",
  ),
  Tool(
      name='Weather',
      func=weather.run,
      description="Look up weather on OpenWeather."
  )  
]
agent_chain = initialize_agent(
    tools=tools,
    llm=chat,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory
)

In [37]:
prompt_task2_v1 = """
Идёт ли сейчас дождь в городе, где находится самый знаменитый фонтан в стиле барокко? 
"""

In [38]:
agent_chain.run(prompt_task2_v1)



> Entering new AgentExecutor chain...
{
    "action": "Search",
    "action_input": "weather famous baroque fountain city"
}
Observation: A fountain, from the Latin "fons" (genitive "fontis"), meaning source or spring, is a decorative reservoir used for discharging water. It is also a structure that jets water into the air for a decorative or dramatic effect.
Fountains were originally purely functional, connected to springs or aqueducts and used to provide drinking water and water for bathing and washing to the residents of cities, towns and villages. Until the late 19th century most fountains operated by gravity, and needed a source of water higher than the fountain, such as a reservoir or aqueduct, to make the water flow or jet into the air.
In addition to providing drinking water, fountains were used for decoration and to celebrate their builders. Roman fountains were decorated with bronze or stone masks of animals or heroes. In the Middle Ages, Moorish and Muslim garden designers

NotFoundError: Unable to find the resource

Когда я запускала в прошлый раз, то почти получила правильный ответ, но по ошибке (он просто определил, что в Риме много фонтанов). Сейчас агент выдает ошибку. Попробуем описать ту самую последовательность действий, которую нужно выполнить агенту.

In [39]:
prompt_task2_v2 = """
Идёт ли сейчас дождь в городе, где находится самый знаменитый фонтан в стиле барокко? 
Чтобы ответить на этот вопрос, нужно:

1. Определить, какой фонтан в стиле барокко является самым знаменитым. 
2. Определить, где находится этот фонтан.
3. Узнать, какая сейчас погода в этом городе.
4. Понять, идет ли там сейчас дождь.
"""

In [40]:
agent_chain.run(prompt_task2_v2)



> Entering new AgentExecutor chain...
{
    "action": "Search",
    "action_input": "самый знаменитый фонтан в стиле барокко"
}
Observation: Could not find [самый знаменитый фонтан в стиле барокко]. Similar: []
Thought:{
    "action": "Final Answer",
    "action_input": "Unfortunately, I cannot provide an answer as the original question is not specific enough. It is unclear which baroque fountain is being referred to and in which city it is located. Without this information, it is impossible to determine whether it is currently raining in the city where the fountain is located. Please provide more specific details so I can assist you better."
}

> Finished chain.


'Unfortunately, I cannot provide an answer as the original question is not specific enough. It is unclear which baroque fountain is being referred to and in which city it is located. Without this information, it is impossible to determine whether it is currently raining in the city where the fountain is located. Please provide more specific details so I can assist you better.'

В прошлый раз удалось получить ответ, в этот раз что-то идет не так. Попробуем иначе.

In [41]:
prompt_task2_v3 = """
Идёт ли сейчас дождь в городе, где находится самый знаменитый фонтан в стиле барокко? 
Чтобы ответить на этот вопрос, нужно:

1. Найти фонтаны в стиле барокко.
2. Определить, какой из этих фонтанов является самым знаменитым. 
3. Определить, где находится этот фонтан.
4. Узнать, какая сейчас погода в этом городе.
5. Понять, идет ли там сейчас дождь.
"""
agent_chain.run(prompt_task2_v3)



> Entering new AgentExecutor chain...
{
    "action": "Search",
    "action_input": "Baroque fountains"
}
Observation: A fountain, from the Latin "fons" (genitive "fontis"), meaning source or spring, is a decorative reservoir used for discharging water. It is also a structure that jets water into the air for a decorative or dramatic effect.
Fountains were originally purely functional, connected to springs or aqueducts and used to provide drinking water and water for bathing and washing to the residents of cities, towns and villages. Until the late 19th century most fountains operated by gravity, and needed a source of water higher than the fountain, such as a reservoir or aqueduct, to make the water flow or jet into the air.
In addition to providing drinking water, fountains were used for decoration and to celebrate their builders. Roman fountains were decorated with bronze or stone masks of animals or heroes. In the Middle Ages, Moorish and Muslim garden designers used fountains to 

'To determine if it is currently raining in the city where the most famous baroque fountain is located, we need to first identify the fountain and its location. Without this information, it is impossible to determine the current weather conditions in that city.'

In [42]:
prompt_task2_v4_1 = """
Какие есть известные фонтаны в стиле барокко?
Где находится самый известный из этих фонтанов?
Какая сейчас погода в этом городе?

Финальный ответ должен быть на русском языке.
"""
agent_chain.run(prompt_task2_v4_1)



> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "известные фонтаны в стиле барокко"
}
```
Observation: Could not find [известные фонтаны в стиле барокко]. Similar: []
Thought:My apologies for the previous response. Here is the updated response:

```json
{
    "action": "Final Answer",
    "action_input": "Один из самых известных фонтанов в стиле барокко - Фонтан Треви, который находится в Риме, Италия. Чтобы узнать погоду в Риме, я могу воспользоваться сервисом OpenWeather."
}
```

> Finished chain.


'Один из самых известных фонтанов в стиле барокко - Фонтан Треви, который находится в Риме, Италия. Чтобы узнать погоду в Риме, я могу воспользоваться сервисом OpenWeather.'

In [44]:
prompt_task2_v4_2 = """
Какие есть известные фонтаны в стиле барокко?
Где находится самый известный из этих фонтанов?
Узнай, какая сейчас погода в этом городе, воспользовавшись сервисом OpenWeather.
Ответь на вопрос: идёт ли сейчас дождь в городе, где находится самый знаменитый фонтан в стиле барокко?

Финальный ответ должен быть на русском языке.
"""
agent_chain.run(prompt_task2_v4_2)



> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "известные фонтаны в стиле барокко"
}
```
Observation: Could not find [известные фонтаны в стиле барокко]. Similar: []
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Один из самых известных фонтанов в стиле барокко - Фонтан Треви, который находится в Риме, Италия. Сейчас я не могу узнать погоду в Риме, так как мне необходимо знать точное время и дату для получения актуальной информации о погоде. Пожалуйста, уточните эту информацию, чтобы я могу помочь вам определить, идет ли в данный момент дождь в Риме."
}
```

> Finished chain.


'Один из самых известных фонтанов в стиле барокко - Фонтан Треви, который находится в Риме, Италия. Сейчас я не могу узнать погоду в Риме, так как мне необходимо знать точное время и дату для получения актуальной информации о погоде. Пожалуйста, уточните эту информацию, чтобы я могу помочь вам определить, идет ли в данный момент дождь в Риме.'

Так как у агента есть память, мы может уточнять предыдущие запросы.

In [45]:
prompt_task2_v4_3 = """
Узнай, какая сейчас погода в Риме, Италия.
"""
agent_chain.run(prompt_task2_v4_3)



> Entering new AgentExecutor chain...
```json
{
    "action": "Weather",
    "action_input": "Rome, Italy"
}
```
Observation: In Rome, Italy, the current weather is as follows:
Detailed status: clear sky
Wind speed: 5.14 m/s, direction: 130°
Humidity: 53%
Temperature: 
  - Current: 25.52°C
  - High: 26.43°C
  - Low: 23.89°C
  - Feels like: 25.51°C
Rain: {}
Heat index: None
Cloud cover: 0%
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Сейчас в Риме, Италия ясное небо. Дождя нет."
}
```

> Finished chain.


'Сейчас в Риме, Италия ясное небо. Дождя нет.'

In [46]:
prompt_task2_v4_4 = """
Дай финальный ответ на вопрос: "идёт ли сейчас дождь в городе, где находится самый знаменитый фонтан в стиле барокко?"
"""
agent_chain.run(prompt_task2_v4_4)




> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Сейчас в Риме, Италия ясное небо. Дождя нет."
}
```

> Finished chain.


'Сейчас в Риме, Италия ясное небо. Дождя нет.'

Если обращаться сразу к поиску, то, скорее всего, ответ будет получить проще, потому что он сможет сразу дать ответ по запросу "самый знаменитый фонтан в стиле барокко", а не выбрасывать ошибку каждый раз.

Таком образом, чтобы получить с помощью ChatGPT ответ на вопрос *идёт ли сейчас дождь в городе, где находится самый знаменитый фонтан в стиле барокко?*, мы можем воспользоваться библиотекой `LangChain`, чтобы написать агента, который сможет по API обращаться к необходимым ресурсам (поиску). Промпты, которые нужно использвать, чтобы получить ответ:
1. Какой самый знаменитый фонтан в стиле барокко?
2. Где находится этот фонтан?
3. Какая сейчас погода в этом городе?
4. Идет ли в этом городе сейчас дождь?

## Задание 3
Используя данные в тексте, необходимо понять, кто из троих полетит в космос. Попробуй получить максимально конкретный ответ. (Текст может быть изменён, но промты должен работать).

*“Мир изменился навсегда, когда в космос была запущена первая искусственная спутница Земли - Спутник-1. Однако еще более захватывающей была история, которая произошла через несколько лет - полет собак в космос. Мы с вами удостоились жить в невероятное время — прямо сейчас проходит новый отбор. В финал вышли три собаки: Белка, Стрелка и Разметка. Мы взяли интервью у организаторов полёта, а именно у Пульсентия Араповича Гагарлёва, и вот, что он сказал: «Все кандидаты очень достойные, но полететь может только одна собака. Мы предъявляем к победителю следующие требования: собака должна быть не выше 30 см в холке, хотя это требование можно не соблюдать, если собака не тяжелее 10 кг, иначе корабль не взлетит, но самое главное, что она должна иметь хвост». Нам удалось выяснить, что Белка — это шпиц 31 см в холке, весом 7 кг и с красивым хвостом. Также сообщают, что Стрелка — джек-рассел-терьер 29 см в холке, весом 9 кг и без хвоста. Загадочная Разметка — это чихуахуа 15 см в холке, 4 кг весом и с коротким хвостом. Полет собак в космос остается одним из наиболее впечатляющих достижений человечества в исследовании космоса. Это наглядно демонстрирует, что не только люди, но и животные могут достичь чего-то такого, что казалось невозможным. Наша цивилизация должна гордиться этим достижением и продолжать исследовать космос, чтобы расширять наши знания и возможности.”*


In [47]:
text_task3 = """
Мир изменился навсегда, когда в космос была запущена первая искусственная спутница Земли - Спутник-1. \
Однако еще более захватывающей была история, которая произошла через несколько лет - полет собак в космос. \
Мы с вами удостоились жить в невероятное время — прямо сейчас проходит новый отбор. \
В финал вышли три собаки: Белка, Стрелка и Разметка. \
Мы взяли интервью у организаторов полёта, а именно у Пульсентия Араповича Гагарлёва, и вот, что он сказал: \
«Все кандидаты очень достойные, но полететь может только одна собака. \
Мы предъявляем к победителю следующие требования: собака должна быть не выше 30 см в холке, \
хотя это требование можно не соблюдать, если собака не тяжелее 10 кг, иначе корабль не взлетит, \
но самое главное, что она должна иметь хвост». \
Нам удалось выяснить, что Белка — это шпиц 31 см в холке, весом 7 кг и с красивым хвостом. \
Также сообщают, что Стрелка — джек-рассел-терьер 29 см в холке, весом 9 кг и без хвоста. \
Загадочная Разметка — это чихуахуа 15 см в холке, 4 кг весом и с коротким хвостом. \
Полет собак в космос остается одним из наиболее впечатляющих достижений человечества в исследовании космоса. \
Это наглядно демонстрирует, что не только люди, но и животные могут достичь чего-то такого, что казалось невозможным. \
Наша цивилизация должна гордиться этим достижением и продолжать исследовать космос, чтобы расширять наши знания и возможности.
"""

In [48]:
prompt_task3_v1 = f"""
На основе текста сделай вывод, какая собака полетит в космос.
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v1)]).content)

Собака, которая полетит в космос, не указана в тексте.


Пробуем дальше.

In [49]:
prompt_task3_v2 = f"""
Какие предъявляются обязательные требования к победителю?
Оцени соответствие параметров каждой собаки каждому из обязательных требований. 
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v2)]).content)

Обязательные требования к победителю:
1. Собака должна быть не выше 30 см в холке - Белка не соответствует этому требованию, так как ее высота в холке составляет 31 см.
2. Собака не должна быть тяжелее 10 кг - Белка и Стрелка соответствуют этому требованию, вес Белки составляет 7 кг, а вес Стрелки - 9 кг.
3. Собака должна иметь хвост - Белка и Разметка соответствуют этому требованию, у них есть хвост, а у Стрелки его нет.

Таким образом, только Белка и Разметка соответствуют всем обязательным требованиям к победителю.


Модель определяет, что Бела и Разметка обе соответствуют параметрам, это **верно**.

In [50]:
prompt_task3_v3 = f"""
Извлеки обязательные требования к собаке, которая может полететь в космос.
Оцени соответствие параметров каждой собаки каждому из обязательных требований. 
Какая собака может полететь в космос?
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v3)]).content)

Обязательные требования к собаке, которая может полететь в космос:
1. Размер не более 30 см в холке или вес не более 10 кг.
2. Наличие хвоста.

Оценка соответствия параметров каждой собаки каждому из обязательных требований:
1. Белка - не соответствует первому требованию, так как ее размер в холке 31 см, но соответствует второму требованию, так как имеет хвост.
2. Стрелка - соответствует первому требованию, так как ее размер в холке 29 см и вес 9 кг, но не соответствует второму требованию, так как не имеет хвоста.
3. Разметка - соответствует обоим требованиям, так как ее размер в холке 15 см, вес 4 кг и имеет короткий хвост.

Исходя из обязательных требований, только Разметка может полететь в космос.


Добавление дополнительного вопроса сбило модель с толку. Ответ **достоверный**, но рассуждения не совсем верные.

In [51]:
prompt_task3_v4 = f"""
Из текста ниже извлеки обязательные требования к собаке, которая может полететь в космос.
Оцени соответствие характеристик каждой из собак обязательным требованиям. 
Какая собака может полететь в космос? 
Если собак, соответствующих обязательным требованиям несколько, то перечисли всех собак, которые могут полететь в космос.
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v4)]).content)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID e6986e5f062765f7bc4523a606a4914c in your message.).


Обязательные требования к собаке, которая может полететь в космос:
1. Размер не более 30 см в холке или вес не более 10 кг.
2. Наличие хвоста.

Собака, которая может полететь в космос - Белка, так как она соответствует обоим обязательным требованиям. Стрелка не соответствует требованию наличия хвоста, а Разметка не соответствует требованию размера.


Выдумывает что-то про требования к размеру и Разметку.

In [52]:
prompt_task3_v5 = f"""
Из текста ниже извлеки требования к победителю.
Извлеки характеристики собак, вышедших в финал.
Сравни характеристики собак-финалистов с обязательными требованиями к победителю. 
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v5)]).content)

Требования к победителю: не выше 30 см в холке (если вес не превышает 10 кг), наличие хвоста.

Характеристики собак-финалистов:
- Белка: шпиц, 31 см в холке, весом 7 кг, с хвостом.
- Стрелка: джек-рассел-терьер, 29 см в холке, весом 9 кг, без хвоста.
- Разметка: чихуахуа, 15 см в холке, весом 4 кг, с коротким хвостом.

Сравнение характеристик собак-финалистов с требованиями к победителю:
- Белка не соответствует требованию по высоте, но весит меньше 10 кг и имеет хвост.
- Стрелка соответствует требованию по высоте, но не имеет хвоста и весит больше 10 кг.
- Разметка соответствует требованию по высоте и имеет хвост, но ее вес не указан.


Модель права только в отношении характеристик Белки.

In [53]:
prompt_task3_v6 = f"""
Из текста ниже извлеки требования к победителю.
Извлеки характеристики собак, вышедших в финал.
Сравни характеристики собак, вышедших в финал, с обязательными требованиями к победителю. 
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v6)]).content)

Требования к победителю: 
- не выше 30 см в холке (если вес не превышает 10 кг)
- должна иметь хвост

Характеристики собак, вышедших в финал:
- Белка: шпиц, 31 см в холке, весом 7 кг, с хвостом
- Стрелка: джек-рассел-терьер, 29 см в холке, весом 9 кг, без хвоста
- Разметка: чихуахуа, 15 см в холке, 4 кг весом, с коротким хвостом

Сравнение характеристик собак, вышедших в финал, с обязательными требованиями к победителю:
- Белка не соответствует требованию по высоте в холке, но ее вес не превышает 10 кг, и у нее есть хвост.
- Стрелка соответствует требованию по высоте в холке, но у нее нет хвоста.
- Разметка соответствует требованию по высоте в холке и имеет хвост.


И еще она не совсем правильно определяем обязательные требования.

In [54]:
prompt_task3_v7 = f"""
Из текста ниже извлеки требования к победителю.
Извлеки характеристики собак, вышедших в финал.
Сравни характеристики собак, вышедших в финал, с обязательными требованиями к победителю. 
Ответь на вопрос, опираясь на эту информацию: какая собака полетит в космос?
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v7)]).content)

Требования к победителю: не выше 30 см в холке (или не тяжелее 10 кг, если выше), должна иметь хвост.

Характеристики собак:
- Белка: шпиц, 31 см в холке, 7 кг, с хвостом.
- Стрелка: джек-рассел-терьер, 29 см в холке, 9 кг, без хвоста.
- Разметка: чихуахуа, 15 см в холке, 4 кг, с коротким хвостом.

Собака, которая полетит в космос, должна иметь хвост, поэтому это будет Белка, шпиц.


In [55]:
prompt_task3_v8 = f"""
Из текста ниже извлеки требования к победителю.
Извлеки характеристики собак, вышедших в финал.
Сравни характеристики собак, вышедших в финал, с обязательными требованиями к победителю. 
Определи, какая собака лучше всего соответствует требованиям и может полететь в космос.
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v8)]).content)

Требования к победителю: не выше 30 см в холке (если вес не превышает 10 кг), наличие хвоста.

Характеристики собак, вышедших в финал:
- Белка: шпиц, 31 см в холке, весом 7 кг, с хвостом.
- Стрелка: джек-рассел-терьер, 29 см в холке, весом 9 кг, без хвоста.
- Разметка: чихуахуа, 15 см в холке, весом 4 кг, с коротким хвостом.

Сравнение характеристик собак, вышедших в финал, с требованиями к победителю:
- Белка соответствует всем требованиям.
- Стрелка не соответствует требованию наличия хвоста.
- Разметка соответствует требованию наличия хвоста, но ее размеры не соответствуют требованию по высоте в холке.

Исходя из этого, лучше всего требованиям соответствует Белка.


In [56]:
prompt_task3_v9 = f"""
Из текста ниже извлеки требования к победителю.
Извлеки характеристики собак, вышедших в финал. 
Определи, какая собака лучше всего соответствует требованиям и может полететь в космос.
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v9)]).content)

Требования к победителю: 
- не выше 30 см в холке (если вес не превышает 10 кг)
- должна иметь хвост

Характеристики собак:
- Белка: шпиц, 31 см в холке, 7 кг, с хвостом
- Стрелка: джек-рассел-терьер, 29 см в холке, 9 кг, без хвоста
- Разметка: чихуахуа, 15 см в холке, 4 кг, с коротким хвостом

Судя по требованиям, лучше всего для полета в космос подходит Белка, так как она соответствует всем требованиям и имеет хвост.


Белка подходит, но не лучше всего. Ответ достоверный, но не совсем точный.

In [57]:
prompt_task3_v10 = f"""
1. Извлеки требования к победителю из текста ниже.
2. Извлеки характеристики собак, вышедших в финал. 
3. Сравни извлеченные характеристики собак с требованиями к победителю и определи, какая собака лучше всего соответствует требованиям и может полететь в космос. 
4. Дай аргументированный ответ на вопрос: "Какая собака полетит в космос?".
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v10)]).content)

1. Требования к победителю: собака должна быть не выше 30 см в холке, хотя это требование можно не соблюдать, если собака не тяжелее 10 кг, иначе корабль не взлетит, но самое главное, что она должна иметь хвост.
2. Характеристики собак:
- Белка - шпиц 31 см в холке, весом 7 кг и с красивым хвостом.
- Стрелка - джек-рассел-терьер 29 см в холке, весом 9 кг и без хвоста.
- Разметка - чихуахуа 15 см в холке, 4 кг весом и с коротким хвостом.
3. Собака, которая лучше всего соответствует требованиям и может полететь в космос - Белка, так как она имеет хвост и не превышает весовые ограничения, хотя несколько больше по размеру, чем требуется.
4. Собака, которая полетит в космос - Белка, так как она соответствует основным требованиям, а именно имеет хвост и не превышает весовые ограничения.


Ответ достоверный, но не совсем правильный.

In [59]:
prompt_task3_v11 = f"""
1. Определи требования к победителю из текста ниже.
2. Извлеки характеристики собак, вышедших в финал. 
3. Оцени соответствие характеристик собак требованиям к победителю. 
4. Определи, какая собака может полететь в космос. Дай аргументированный ответ на вопрос. \
Если есть несколько собак, соответствующих требованиям, то выбери собаку, которая лучше соответствует требованиям.
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v11)]).content)

1. Требования к победителю: собака должна быть не выше 30 см в холке, хотя это требование можно не соблюдать, если собака не тяжелее 10 кг, иначе корабль не взлетит, но самое главное, что она должна иметь хвост.
2. Характеристики собак:
- Белка: шпиц, 31 см в холке, весом 7 кг, с хвостом.
- Стрелка: джек-рассел-терьер, 29 см в холке, весом 9 кг, без хвоста.
- Разметка: чихуахуа, 15 см в холке, весом 4 кг, с коротким хвостом.
3. Белка соответствует требованиям к победителю, кроме того, что она немного выше 30 см в холке. Стрелка не соответствует требованию о наличии хвоста. Разметка соответствует всем требованиям.
4. Собака, которая может полететь в космос - Разметка. Она соответствует всем требованиям к победителю.


Получен **верный** ответ.

In [61]:
prompt_task3_v12 = f"""
1. Определи требования к победителю из текста ниже.
2. Извлеки характеристики собак, вышедших в финал. 
3. Оцени соответствие характеристик собак требованиям к победителю. 
4. Определи, какая собака полетит в космос. Дай аргументированный ответ на вопрос. \
Если есть несколько собак, соответствующих требованиям, то выбери собаку, которая лучше соответствует требованиям.
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v12)]).content)

1. Требования к победителю: собака должна быть не выше 30 см в холке, хотя это требование можно не соблюдать, если собака не тяжелее 10 кг, иначе корабль не взлетит, но самое главное, что она должна иметь хвост.
2. Характеристики собак:
- Белка: шпиц, 31 см в холке, весом 7 кг, с красивым хвостом.
- Стрелка: джек-рассел-терьер, 29 см в холке, весом 9 кг, без хвоста.
- Разметка: чихуахуа, 15 см в холке, 4 кг весом, с коротким хвостом.
3. Собака Белка не соответствует требованию по высоте в холке, но ее вес не превышает 10 кг, а также она имеет хвост. Собака Стрелка не имеет хвоста, а собака Разметка соответствует всем требованиям.
4. Собака, которая полетит в космос, - Разметка. Она соответствует всем требованиям к победителю.


Поменяем текст и посмотрим, работает ли промпт.

In [63]:
prompt_task3_v12_2 = f"""
1. Определи требования к победителю из текста ниже.
2. Извлеки характеристики собак, вышедших в финал. 
3. Оцени соответствие характеристик собак требованиям к победителю. 
4. Определи, какая собака полетит в космос. Дай аргументированный ответ на вопрос. \
Если есть несколько собак, соответствующих требованиям, то выбери собаку, которая лучше соответствует требованиям.
---
Мир изменился навсегда, когда в космос была запущена первая искусственная спутница Земли - Спутник-1. \
Однако еще более захватывающей была история, которая произошла через несколько лет - полет собак в космос. \
Мы с вами удостоились жить в невероятное время — прямо сейчас проходит новый отбор. \
В финал вышли три собаки: Белка, Стрелка и Разметка. \
Мы взяли интервью у организаторов полёта, а именно у Пульсентия Араповича Гагарлёва, и вот, что он сказал: \
«Все кандидаты очень достойные, но полететь может только одна собака. \
Мы предъявляем к победителю следующие требования: собака должна быть не выше 30 см в холке, \
хотя это требование можно не соблюдать, если собака не тяжелее 10 кг, иначе корабль не взлетит, \
но самое главное, что она должна иметь хвост». \
Нам удалось выяснить, что Белка — это шпиц 25 см в холке, весом 7 кг и с красивым хвостом. \
Также сообщают, что Стрелка — джек-рассел-терьер 29 см в холке, весом 11 кг и без хвоста. \
Загадочная Разметка — это хаски 55 см в холке, 25 кг весом и с хвостом. \
Полет собак в космос остается одним из наиболее впечатляющих достижений человечества в исследовании космоса. \
Это наглядно демонстрирует, что не только люди, но и животные могут достичь чего-то такого, что казалось невозможным. \
Наша цивилизация должна гордиться этим достижением и продолжать исследовать космос, чтобы расширять наши знания и возможности.
---
"""
print(chat([HumanMessage(content=prompt_task3_v12_2)]).content)

1. Требования к победителю: собака должна быть не выше 30 см в холке (если вес не превышает 10 кг), иметь хвост.
2. Характеристики собак:
- Белка: шпиц, 25 см в холке, вес 7 кг, с хвостом.
- Стрелка: джек-рассел-терьер, 29 см в холке, вес 11 кг, без хвоста.
- Разметка: хаски, 55 см в холке, вес 25 кг, с хвостом.
3. Собака Белка соответствует требованиям к победителю, так как она не выше 30 см в холке, весит меньше 10 кг и имеет хвост. Собака Стрелка не соответствует требованию наличия хвоста, а собака Разметка не соответствует требованию размера и веса.
4. Собака Белка полетит в космос, так как она единственная соответствует всем требованиям к победителю.


In [64]:
prompt_task3_v13 = f"""
Дай аргументированный ответ на вопрос: "Какая собака может полететь в космос?", опираясь на данные из текста ниже.
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v13)]).content)

Собака, которая может полететь в космос, должна быть не выше 30 см в холке и не тяжелее 10 кг, а также иметь хвост. Из трех кандидатов на полет - Белка, Стрелка и Разметка - только Белка соответствует всем требованиям, включая наличие хвоста. Поэтому Белка может полететь в космос.


In [65]:
prompt_task3_v16 = f"""
Сравни характеристики собак-финалистов с требованиями, предъявляемыми победителям.
Дай аргументированный ответ на вопрос: "Какая собака может полететь в космос?", опираясь на данные из текста ниже.
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v16)]).content)

Требования, предъявляемые к победителю полета собаки в космос, включают в себя не только размеры и вес, но и наличие хвоста. Из трех финалистов только Белка соответствует всем требованиям, включая наличие хвоста. Стрелка не имеет хвоста, а Разметка слишком маленькая и легкая для полета. Следовательно, только Белка может полететь в космос согласно требованиям, предъявляемым к победителю. 

Ответ на вопрос "Какая собака может полететь в космос?" - только собака, которая соответствует всем требованиям, предъявляемым к победителю, включая наличие хвоста. В данном случае это Белка.


In [66]:
prompt_task3_v17 = f"""
Дай аргументированный ответ на вопрос: "Какая собака может полететь в космос?", опираясь на данные из текста ниже.
Пример рассуждений: 
Обязательные требования к собаке:
- высота в холке не выше 20 см
- вес не больше 15 кг
Лайка - это собака породы хаски, 52 см в холке, весит 20 кг. Лайка не соответствует требованиям, не может полететь в космос.
Жучка - это собака породы корги, 25 см в холке, весит 15 кг. Жучка не соответствует требованиям, не может полететь в космос.
Шерлок - это такса, 19 см в холке, весит 8 кг. Шерлок соответствует требованиям, может полететь в космос.
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v17)]).content)

Собака, которая может полететь в космос, должна соответствовать следующим требованиям: не выше 30 см в холке, не тяжелее 10 кг и иметь хвост. Из трех собак, упомянутых в тексте, только Белка соответствует этим требованиям. Она шпиц, 31 см в холке, весом 7 кг и с красивым хвостом. Стрелка и Разметка не могут полететь в космос, так как не соответствуют требованиям.


In [67]:
prompt_task3_v18 = f"""
Дай аргументированный ответ на вопрос: "Какая собака может полететь в космос?", опираясь на данные из текста ниже.
Определи обязательные и необязательные требования к победителю.
Извлеки характеристики собак, вышедших в финал. 
Оцени соответствие характеристик собак обязательным требованиям к победителю. 
Определи, какие собаки могут полететь в космос, опираясь на извлеченные из текста данные. \
Подходящих собак может не быть, может быть одна, может быть несколько.
Если собак, соответствующих требованиям, несколько, выбери собаку, которая лучше соответствует требованиям.

Пример рассуждений: 
Требования к победителю: высота в холке не выше 20 см, если вес больше 15 кг.
Лайка - это собака породы хаски, 52 см в холке, весит 20 кг. Лайка выше 20 см и весит больше 15 кг, поэтому она не может полететь в космос.
Жучка - это собака породы корги, 25 см в холке, весит 14 кг. Жучка выше 20 см, но ее вес не превышает 15 кг, поэтому она может полететь в космос.
Шерлок - это такса, 19 см в холке, весит 8 кг. Шерлок соответствует всем требованиям, может полететь в космос.
Жучка и Шерлок могут полететь в космос, но Шерлок лучше соответствует всем требованиям. Шерлок полетит в космос.
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v18)]).content)

Обязательные требования к победителю: собака должна иметь хвост, не тяжелее 10 кг, не выше 30 см в холке.
Необязательные требования к победителю: отсутствуют.

Характеристики собак:
- Белка: шпиц, 31 см в холке, 7 кг, с хвостом.
- Стрелка: джек-рассел-терьер, 29 см в холке, 9 кг, без хвоста.
- Разметка: чихуахуа, 15 см в холке, 4 кг, с коротким хвостом.

Собака, которая может полететь в космос - Белка. Она соответствует всем обязательным требованиям к победителю и имеет хвост. Стрелка не соответствует обязательному требованию наличия хвоста, а Разметка не соответствует требованию не тяжелее 10 кг.


In [68]:
prompt_task3_v19 = f"""
Дай аргументированный ответ на вопрос: "Какая собака может полететь в космос?", опираясь на данные из текста ниже.
Определи обязательные и желательные требования к победителю.
Извлеки характеристики собак, вышедших в финал. 
Оцени соответствие характеристик собак обязательным требованиям к победителю. 
Определи, какие собаки могут полететь в космос, опираясь на извлеченные из текста данные. \
Подходящих собак может не быть, может быть одна, может быть несколько.
Если собак, соответствующих требованиям, несколько, выбери собаку, которая лучше соответствует требованиям.
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v19)]).content)

Собака, которая может полететь в космос, должна соответствовать следующим обязательным требованиям: не превышать 30 см в холке или не весить более 10 кг, а также иметь хвост. Желательными требованиями являются хорошее здоровье, выносливость и спокойный характер.

Из текста можно извлечь следующие характеристики собак, вышедших в финал: Белка - шпиц 31 см в холке, весом 7 кг и с красивым хвостом; Стрелка - джек-рассел-терьер 29 см в холке, весом 9 кг и без хвоста; Разметка - чихуахуа 15 см в холке, 4 кг весом и с коротким хвостом.

Исходя из обязательных требований, только Белка может полететь в космос, так как она соответствует всем требованиям. Стрелка не имеет хвоста, а Разметка слишком легка и маленькая. 

Таким образом, Белка - единственная собака, которая может полететь в космос, так как она соответствует всем обязательным требованиям.


Еще раз запустим промпт, который дал нам правильный ответ.

In [69]:
prompt_task3_v20 = f"""
1. Определи требования к победителю из текста ниже.
2. Извлеки характеристики собак, вышедших в финал. 
3. Оцени соответствие характеристик собак требованиям к победителю. 
4. Определи, какая собака может полететь в космос. Дай аргументированный ответ на вопрос. \
Если есть несколько собак, соответствующих требованиям, то выбери собаку, которая лучше соответствует требованиям.
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v20)]).content)

1. Требования к победителю: собака должна быть не выше 30 см в холке, хотя это требование можно не соблюдать, если собака не тяжелее 10 кг, иначе корабль не взлетит, но самое главное, что она должна иметь хвост.
2. Характеристики собак:
- Белка: шпиц, 31 см в холке, весом 7 кг, с красивым хвостом.
- Стрелка: джек-рассел-терьер, 29 см в холке, весом 9 кг, без хвоста.
- Разметка: чихуахуа, 15 см в холке, 4 кг весом, с коротким хвостом.
3. Собака Белка не соответствует требованию по высоте в холке, но ее вес меньше 10 кг и она имеет хвост. Собака Стрелка не имеет хвоста, а Разметка соответствует всем требованиям.
4. Собака, которая может полететь в космос - Разметка, так как она соответствует всем требованиям: не выше 30 см в холке, весом менее 10 кг и имеет хвост.


In [92]:
prompt_task3_v21 = f"""
Дай аргументированный ответ на вопрос: "Какая собака полетит в космос?", опираясь на информацию из текста ниже. \
Ответ должен включать цепочку рассуждений по шагам.
---
{text_task3}
---
"""
print(chat([HumanMessage(content=prompt_task3_v21)]).content)

Из текста можно сделать вывод, что собака, которая может полететь в космос, должна быть не выше 30 см в холке и не тяжелее 10 кг, а также иметь хвост. Из трех кандидатов только у Белки есть хвост, но она превышает требование по высоте в холке на 1 см. Стрелка не имеет хвоста, а Разметка соответствует всем требованиям. Следовательно, Разметка может полететь в космос.


## Задание 4
В ООО «Ромашка» 13 мая 2023 года прошло собрание участников, где они голосованием одобрили заключение крупной сделки с ООО «Любит-не любит» по покупке 7000 га земли в Подмосковье. Участниками были: Сюзанна Ивановна Первова, Лаврентий Иосифович Сухумский, Серсея Михайловна Демагогина и Шевелина Андреевна Поршнева, которая была председателем собрания. Все участники проголосовали “За”. Пусть ChatGPT составит протокол. 

In [70]:
prompt_task4_v1 = """
Составь протокол собрания участников.
---
В ООО «Ромашка» 13 мая 2023 года прошло собрание участников, где они голосованием одобрили заключение крупной сделки \
с ООО «Любит-не любит» по покупке 7000 га земли в Подмосковье. \
Участниками были: Сюзанна Ивановна Первова, Лаврентий Иосифович Сухумский, Серсея Михайловна Демагогина \
и Шевелина Андреевна Поршнева, которая была председателем собрания. Все участники проголосовали “За”.
---
"""
print(chat([HumanMessage(content=prompt_task4_v1)]).content)

Протокол собрания участников ООО «Ромашка» от 13 мая 2023 года.

Участники: 
1. Сюзанна Ивановна Первова
2. Лаврентий Иосифович Сухумский
3. Серсея Михайловна Демагогина
4. Шевелина Андреевна Поршнева (председатель)

Повестка дня: 
1. Одобрение заключения сделки с ООО «Любит-не любит» по покупке 7000 га земли в Подмосковье.

Председатель собрания Шевелина Андреевна Поршнева открыла заседание в 10:00 и предложила участникам приступить к рассмотрению повестки дня.

Участники обсудили предложенную сделку и проголосовали за ее одобрение. Все участники проголосовали «За».

Председатель собрания Шевелина Андреевна Поршнева закрыла заседание в 10:30.

Подписи участников:
1. Сюзанна Ивановна Первова
2. Лаврентий Иосифович Сухумский
3. Серсея Михайловна Демагогина
4. Шевелина Андреевна Поршнева (председатель)


В принципе, уже похоже на протокол. ChatGPT сам выдумал время проведения, но оно должно быть в протоколе.

In [71]:
prompt_task4_v2 = """
Составь протокол общего собрания участников.
Организация: ООО «Ромашка»
Дата: 13 мая 2023 года
Повестка: заключение крупной сделки с ООО «Любит-не любит» по покупке 7000 га земли в Подмосковье
Участники: Сюзанна Ивановна Первова, Лаврентий Иосифович Сухумский, Серсея Михайловна Демагогина, Шевелина Андреевна Поршнева
Председатель собрания: Шевелина Андреевна Поршнева
Все участники проголосовали "За"
"""
print(chat([HumanMessage(content=prompt_task4_v2)]).content)

Протокол общего собрания участников ООО «Ромашка» от 13 мая 2023 года.

Собрание было созвано для обсуждения заключения крупной сделки с ООО «Любит-не любит» по покупке 7000 га земли в Подмосковье.

В собрании приняли участие Сюзанна Ивановна Первова, Лаврентий Иосифович Сухумский, Серсея Михайловна Демагогина и Шевелина Андреевна Поршнева.

Председательствовала на собрании Шевелина Андреевна Поршнева.

После обсуждения предложения о заключении сделки, все участники проголосовали "За".

Решение было принято единогласно.

Собрание завершилось в 15:00.

Подписи участников:

Шевелина Андреевна Поршнева - председатель собрания

Сюзанна Ивановна Первова - участник

Лаврентий Иосифович Сухумский - участник

Серсея Михайловна Демагогина - участник


In [74]:
prompt_task4_v3 = """
Составь протокол общего собрания участников на основе текста внутри разделителей.
Протокол обязательно должен содержать дату проведения, сведения о лицах, принявших участие в собрании, \
информацию о кворуме, повестку дня, результат голосования по повестке, решение по повестке, \
а также блок с подписями.
---
В ООО «Ромашка» 13 мая 2023 года прошло собрание участников, где они голосованием одобрили заключение крупной сделки \
с ООО «Любит-не любит» по покупке 7000 га земли в Подмосковье. \
Участниками были: Сюзанна Ивановна Первова, Лаврентий Иосифович Сухумский, Серсея Михайловна Демагогина \
и Шевелина Андреевна Поршнева, которая была председателем собрания. Все участники проголосовали “За”.
---
"""
print(chat([HumanMessage(content=prompt_task4_v3)]).content)

Протокол общего собрания участников ООО «Ромашка» от 13 мая 2023 года.

Участники собрания: 
1. Первова Сюзанна Ивановна
2. Сухумский Лаврентий Иосифович
3. Демагогина Серсея Михайловна
4. Поршнева Шевелина Андреевна (председатель собрания)

Кворум на собрании имелся, так как присутствовали все участники.

Повестка дня:
1. Рассмотрение вопроса о заключении сделки с ООО «Любит-не любит» по покупке 7000 га земли в Подмосковье.

Результат голосования: 
Все участники собрания проголосовали «За» заключение сделки с ООО «Любит-не любит» по покупке 7000 га земли в Подмосковье.

Решение по повестке дня:
Собрание участников ООО «Ромашка» одобрило заключение сделки с ООО «Любит-не любит» по покупке 7000 га земли в Подмосковье.

Подписи участников собрания:
1. Первова С.И. - _______________________
2. Сухумский Л.И. - _______________________
3. Демагогина С.М. - _______________________
4. Поршнева Ш.А. - _______________________ (подпись председателя собрания)


## Задание 5
Я не юрист, но хочу понять, о чём тут идет речь. Помоги мне в этом с помощью ChatGPT. Критерий выполнения: получен пересказ и содержание текста понятно обычному человеку.
*“Исторически считалось, что зачет не должен допускаться, если требование ответчика является спорным и может привести к затягиванию процесса (Liqudi cum illiquido non fit compensatio). Отсюда гипотетически можно было вывести юридическую разнородность в дихотомии регулятивных и охранительных отношений. Мы действительно должны разграничивать долг и ответственность. И то и другое суть действия, в том числе выражающиеся в уплате денег, т.е. обязательства. Однако требования совершения таких действий подчинены разному регулированию. Первоначально считалось, что эти требования отличаются по степени их спорности: долг менее спорен, а ответственность более спорна. Отсюда и прежняя арбитражная практика, которая, например, не допускала зачета уплаты цены требованием о неустойке. В части процентов за пользование чужими денежными средствами и неустойки данная практика была пересмотрена еще ранее, и Пленум закрепляет этот подход. Требования об уплате процентов за пользование чужими денежными средствами и неустойки могут быть зачтены против требования об уплате долга. Степень спорности этих притязаний может быть совершенно разной, размер и основание неустойки могут оказаться менее спорными, а наличие долга или его размер может обладать весьма высокой степенью спорности. Два противостоящих долга могут иметь разную спорность. Все может быть спорным.”*


In [75]:
text_task5 = """
Исторически считалось, что зачет не должен допускаться, если требование ответчика является спорным \
и может привести к затягиванию процесса (Liqudi cum illiquido non fit compensatio). \
Отсюда гипотетически можно было вывести юридическую разнородность в дихотомии регулятивных и охранительных отношений. \
Мы действительно должны разграничивать долг и ответственность. \
И то и другое суть действия, в том числе выражающиеся в уплате денег, т.е. обязательства. \
Однако требования совершения таких действий подчинены разному регулированию. \
Первоначально считалось, что эти требования отличаются по степени их спорности: долг менее спорен, \
а ответственность более спорна. Отсюда и прежняя арбитражная практика, которая, например, не допускала зачета \
уплаты цены требованием о неустойке. В части процентов за пользование чужими денежными средствами и неустойки \
данная практика была пересмотрена еще ранее, и Пленум закрепляет этот подход. \
Требования об уплате процентов за пользование чужими денежными средствами и неустойки \
могут быть зачтены против требования об уплате долга. Степень спорности этих притязаний может быть совершенно разной, \
размер и основание неустойки могут оказаться менее спорными, а наличие долга или его размер может обладать весьма \
высокой степенью спорности. Два противостоящих долга могут иметь разную спорность. Все может быть спорным.
"""

In [76]:
prompt_task5_v1 = f"""
Перескажи текст ниже.
---
{text_task5}
---
"""

In [77]:
print(chat([HumanMessage(content=prompt_task5_v1)]).content)

Раньше считалось, что зачет не должен быть разрешен, если требование ответчика спорное и может затянуть процесс. Однако, долг и ответственность должны быть разделены, так как они оба являются обязательствами, но подчинены разному регулированию. Раньше считалось, что долг менее спорен, а ответственность более спорна, но это не всегда так. Сейчас требования об уплате процентов за пользование чужими денежными средствами и неустойки могут быть зачтены против требования об уплате долга, независимо от степени их спорности.


In [78]:
prompt_task5_v2 = f"""
Перескажи текст ниже с разъяснением юридических терминов. 
---
{text_task5}
---
"""
print(chat([HumanMessage(content=prompt_task5_v2)]).content)

В юридическом сообществе долг и ответственность рассматриваются как разные виды обязательств. Долг - это обязательство уплатить деньги, а ответственность - это обязательство возместить убытки, которые могут быть вызваны нарушением договора. Ранее считалось, что долг менее спорен, чем ответственность, и поэтому уплата долга не могла быть зачтена против требования об уплате неустойки. Однако, сейчас этот подход пересмотрен, и требования об уплате процентов за пользование чужими денежными средствами и неустойки могут быть зачтены против требования об уплате долга. Спорность этих притязаний может быть разной, и размер и основание неустойки могут оказаться менее спорными, чем наличие долга или его размер. Два противостоящих долга могут иметь разную степень спорности.


In [79]:
prompt_task5_v3 = f"""
Перескажи текст ниже, объясняя юридические термины. 
---
{text_task5}
---
"""
print(chat([HumanMessage(content=prompt_task5_v3)]).content)

Текст рассказывает о юридической практике зачета требований в судебных процессах. Ранее считалось, что зачет не должен допускаться, если требование ответчика является спорным и может затянуть процесс. Однако, существует различие между долгом и ответственностью, которые оба являются обязательствами и выражаются в уплате денег. Требования к уплате процентов за пользование чужими денежными средствами и неустойки могут быть зачтены против требования об уплате долга, но степень спорности этих требований может быть разной. Различные долги могут иметь разную степень спорности, и все требования могут быть спорными.


In [80]:
prompt_task5_v4 = f"""
Подробно перескажи текст ниже так, чтобы он был понятен человеку, не разбирающемуся в юридических вопросах.
---
{text_task5}
---
"""
print(chat([HumanMessage(content=prompt_task5_v4)]).content)

Раньше считалось, что если требование ответчика спорное и может затянуть процесс, то зачет не должен быть допущен. Это связано с различием между долгом и ответственностью, которые оба являются обязательствами, но регулируются по-разному. Раньше считалось, что долг менее спорен, а ответственность более спорна. Поэтому, например, уплата цены не могла быть зачтена против требования о неустойке. Однако, в случае процентов за пользование чужими денежными средствами и неустойки, этот подход был пересмотрен. Теперь требования об уплате процентов за пользование чужими денежными средствами и неустойки могут быть зачтены против требования об уплате долга, даже если размер и основание неустойки менее спорны, чем наличие долга или его размер. Все может быть спорным.


In [81]:
prompt_task5_v5 = f"""
Подробно перескажи текст ниже с объяснением юридических терминов так, чтобы он был понятен человеку, не являющемуся юристом.
---
{text_task5}
---
"""
print(chat([HumanMessage(content=prompt_task5_v5)]).content)

Раньше считалось, что если у ответчика есть спорное требование, то зачет не должен быть допущен, чтобы не затягивать процесс. Это связано с тем, что долг и ответственность - это разные вещи, хотя оба они выражаются в уплате денег. Требования к уплате денег могут быть регулируемы по-разному. Раньше считалось, что долг менее спорен, а ответственность более спорна. Поэтому арбитражная практика не допускала зачета уплаты цены требованием о неустойке. Однако, в случае процентов за пользование чужими денежными средствами и неустойки, этот подход был пересмотрен. Теперь требования об уплате процентов за пользование чужими денежными средствами и неустойки могут быть зачтены против требования об уплате долга. Спорность требований может быть разной, и размер и основание неустойки могут оказаться менее спорными, чем наличие долга или его размер. Два противостоящих долга могут иметь разную степень спорности.


In [84]:
prompt_task5_v6 = f"""
Подробно перескажи текст ниже так, чтобы он был понятен обычному человеку. При необходимости объясни юридические термины.
---
{text_task5}
---
"""
print(chat([HumanMessage(content=prompt_task5_v6)]).content)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)).


Раньше считалось, что если требование, которое должен выплатить ответчик, вызывает споры и может затянуть судебный процесс, то зачет этого требования не допускается. Это связано с тем, что долг и ответственность - это разные вещи, хотя оба они связаны с выплатой денег. Однако, эти требования регулируются по-разному. Раньше считалось, что долг менее спорен, а ответственность более спорна. Поэтому, например, не допускался зачет уплаты цены за товар требованием об уплате неустойки. Но сейчас практика изменилась и требования об уплате процентов за пользование чужими деньгами и неустойки могут быть зачтены против требования об уплате долга. Размер и основание неустойки могут быть менее спорными, чем наличие долга или его размер. Два противостоящих долга могут иметь разную степень спорности. Все может быть спорным.


In [85]:
prompt_task5_v7 = f"""
Подробно перескажи текст ниже так, чтобы он был понятен обычному человеку. При необходимости объясни термины.
---
{text_task5}
---
"""
print(chat([HumanMessage(content=prompt_task5_v7)]).content)

Раньше считалось, что если требование ответчика спорное, то зачет не должен допускаться, чтобы не затягивать процесс. Но это приводило к разнородности в правилах. Долг и ответственность - это обязательства, которые могут выражаться в уплате денег. Но требования к этим обязательствам регулируются по-разному. Раньше считалось, что долг менее спорен, а ответственность более спорна. Поэтому раньше не допускали зачета уплаты цены требованием о неустойке. Но сейчас практика изменилась и требования об уплате процентов за пользование чужими денежными средствами и неустойки могут быть зачтены против требования об уплате долга. Размер и основание неустойки могут быть менее спорными, чем наличие долга или его размер. Два противостоящих долга могут иметь разную степень спорности. Все может быть спорным.


## Задание 6
Подбери промт, позволяющий решить задачу и ответить на вопросы как живой человек. Критерий выполнения: получены ответы на вопросы задачи, есть аргументы, ответ достоверный.

*“Киреев, которому срочно понадобились деньги, попросил у Иванова взаймы 100 тыс. рублей на два месяца. Иванов согласился дать в долг эту сумму, но с условием, что Киреев передаст ему в залог имеющуюся у последнего в собственности картину художника Васнецова. Соглашение о займе было удостоверено распиской, которую Иванов дал Кирееву. Письменного соглашения о залоге заключено не было. Стоимость картины была оценена в 200 тыс. рублей. Через три месяца Киреев принес Иванову взятые взаймы 100 тыс. рублей, однако последний отказался принять деньги, заявив, что, поскольку Киреев просрочил возврат долга, картина стала его собственностью.  Как решить спор? Изменится ли решение (и если «да», то как), если соглашение о залоге было оформлено в письменную форму?”*


In [82]:
prompt_task6_v1 = """
Киреев, которому срочно понадобились деньги, попросил у Иванова взаймы 100 тыс. рублей на два месяца. \
Иванов согласился дать в долг эту сумму, но с условием, что Киреев передаст ему в залог имеющуюся \
у последнего в собственности картину художника Васнецова. Соглашение о займе было удостоверено распиской, \
которую Иванов дал Кирееву. Письменного соглашения о залоге заключено не было. \
Стоимость картины была оценена в 200 тыс. рублей. Через три месяца Киреев принес Иванову взятые взаймы 100 тыс. рублей, \
однако последний отказался принять деньги, заявив, что, поскольку Киреев просрочил возврат долга, картина стала \
его собственностью. Как решить спор? Изменится ли решение (и если «да», то как), \
если соглашение о залоге было оформлено в письменную форму?
"""

In [83]:
print(chat([HumanMessage(content=prompt_task6_v1)]).content)

Согласно законодательству РФ, залогом могут быть предметы, которые могут быть переданы из рук в руки. Картина является таким предметом, поэтому она может быть передана в залог. Однако, в данном случае, письменного соглашения о залоге не было заключено, поэтому Иванов не имеет права на картину.

Согласно условиям соглашения о займе, Киреев должен был вернуть деньги в течение двух месяцев. Однако, он вернул их только через три месяца. Это означает, что он просрочил возврат долга. В этом случае, Иванов имеет право требовать у Киреева не только возврата долга, но и уплаты процентов за просрочку.

Если бы соглашение о залоге было оформлено в письменную форму, то Иванов имел бы право на картину в случае просрочки возврата долга. Однако, в данном случае, он не имеет права на картину, так как соглашение о залоге не было оформлено в письменную форму.

Таким образом, Иванов не имеет права на картину, но имеет право требовать у Киреева возврата долга и уплаты процентов за просрочку.


Ответ уже выглядит достоверно и аргументированно.

In [87]:
prompt_task6_v2 = """
Ты должен дать пользователю консультацию по юридическому вопросу. 
Определи суть спорной ситуации, описанной в тексте ниже. 
Ответь на следующие вопросы:
- Как решить спор?
- Изменится ли решение, если соглашение о залоге было оформлено в письменной форме?
- Если "да", то как решение изменится?
---
Киреев, которому срочно понадобились деньги, попросил у Иванова взаймы 100 тыс. рублей на два месяца. \
Иванов согласился дать в долг эту сумму, но с условием, что Киреев передаст ему в залог имеющуюся \
у последнего в собственности картину художника Васнецова. Соглашение о займе было удостоверено распиской, \
которую Иванов дал Кирееву. Письменного соглашения о залоге заключено не было. \
Стоимость картины была оценена в 200 тыс. рублей. Через три месяца Киреев принес Иванову взятые взаймы 100 тыс. рублей, \
однако последний отказался принять деньги, заявив, что, поскольку Киреев просрочил возврат долга, картина стала \
его собственностью. 
---
"""
print(chat([HumanMessage(content=prompt_task6_v2)]).content)

Суть спорной ситуации заключается в том, что Иванов дал в долг Кирееву 100 тыс. рублей под залог картину художника Васнецова, но не заключил письменного соглашения о залоге. Киреев вернул долг через три месяца, но Иванов отказался принять деньги, заявив, что картина стала его собственностью из-за просрочки возврата долга.

Для решения спора необходимо обратиться к законодательству о залоге и договору займа. Если в расписке о займе было указано условие о залоге, то Иванов имеет право на картину в случае просрочки возврата долга. Однако, без письменного соглашения о залоге, Иванов может столкнуться с трудностями в доказательстве своих прав на картину.

Если бы соглашение о залоге было оформлено в письменной форме, то Иванов имел бы более убедительные доказательства своих прав на картину. В этом случае, решение спора могло бы измениться в пользу Иванова.

В целом, для решения подобных споров рекомендуется заключать письменные соглашения о залоге и договоры займа, чтобы избежать недоразуме

Выглядит логично.